In [1]:
from urllib2 import urlopen
from json import load
import pandas as pd
from pandas.io.json import json_normalize
import itertools

In [2]:
##api_key = 'MDIzNDI5NDMzMDE0NTkwMTI5ODY1NzQ4Ng000'

In [3]:
url = 'http://api.npr.org/query?apiKey=MDIzNDI5NDMzMDE0NTkwMTI5ODY1NzQ4Ng000&numResults=50&format=json&id=1002'

In [4]:
response = urlopen(url)
json_obj = load(response)

In [5]:
##the next several cells are test code to practice mapping to the information we want in the df -- commented out for now

In [6]:
'''
for story in json_obj['list']['story']:

    print 'ID: ' + story['id']
    
    print 'TITLE: ' + story['title']['$text']

    print 'DATE: ' + story['storyDate']['$text'] + '\n'
'''

"\nfor story in json_obj['list']['story']:\n\n    print 'ID: ' + story['id']\n    \n    print 'TITLE: ' + story['title']['$text']\n\n    print 'DATE: ' + story['storyDate']['$text'] + '\n'\n"

In [7]:
'''for story in json_obj['list']['story']:
    for parent in story['parent']:
        print story['id'], parent['title']['$text']
'''

"for story in json_obj['list']['story']:\n    for parent in story['parent']:\n        print story['id'], parent['title']['$text']\n"

In [8]:
'''for story in json_obj['list']['story']:
    print story['id'], story['text']['paragraph']
'''
'''      
for story in json_obj['list']['story']:
    for text in story['text']['paragraph']:
        print story['id'], text['$text']
'''

"      \nfor story in json_obj['list']['story']:\n    for text in story['text']['paragraph']:\n        print story['id'], text['$text']\n"

In [9]:
## Build the data frame by creating a list of dictionaries, then converting the list of dictionaries into a data frame

##First data frame: one row per distinct story category

dicts_list = []

In [10]:
for story in json_obj['list']['story']:
    for parent in story['parent']:
        d = {
            'id': story['id'],
            'title': story['title']['$text'].encode('ascii'),
            'category': parent['title']['$text'].encode('ascii')    
        }
        dicts_list.append(d)

In [11]:
df = pd.DataFrame(dicts_list, columns=('id', 'title', 'category'))

In [12]:
df.head(5)

id                                              title  \
0  481776539  Obama: 'An Attack On Any American ... Is An At...   
1  481776539  Obama: 'An Attack On Any American ... Is An At...   
2  481776539  Obama: 'An Attack On Any American ... Is An At...   
3  481776539  Obama: 'An Attack On Any American ... Is An At...   
4  481776539  Obama: 'An Attack On Any American ... Is An At...   

                category  
0                   News  
1  Home Page Top Stories  
2         mass shootings  
3                Orlando  
4                America

In [25]:
len(df)

591

In [31]:
len(df['category'].unique())

208

In [13]:
##Second data frame: one row per story, with one column containing a list of all the story's categories

dicts_reshape = []

In [14]:
for story in json_obj['list']['story']:
    categories_list = []
    d = {
        'id': story['id'],
        'title': story['title']['$text'].encode('ascii')
        }
    for parent in story['parent']:
        category = parent['title']['$text'].encode('ascii')
        categories_list.append(category)   
        d['category'] = categories_list
        d['top_category'] = []
    dicts_reshape.append(d)

In [15]:
df_reshape = pd.DataFrame(dicts_reshape, columns=('id', 'title', 'category', 'top_category'))

In [32]:
df_reshape.head()

id                                              title  \
0  481776539  Obama: 'An Attack On Any American ... Is An At...   
1  481766394       50 Dead, 53 Hospitalized In Orlando Shooting   
2  481702569      This Puzzle Packs Syllables Into Small Spaces   
3  481679101  To Fight Floundering Attendance, SeaWorld Turn...   
4  481620159  Latinos And American Identity In A Time Of Tru...   

                                            category top_category  
0  [News, Home Page Top Stories, mass shootings, ...           []  
1  [U.S., Weekend Edition Sunday for June 12, 201...           []  
2  [Games & Humor, Weekend Edition Sunday for Jun...           []  
3  [Business, Weekend Edition Sunday for June 12,...           []  
4  [Politics, News, Weekend Edition Sunday for Ju...           []

In [17]:
df_reshape['category'][0]

['News',
 'Home Page Top Stories',
 'mass shootings',
 'Orlando',
 'America',
 'Barack Obama',
 'The Two-Way',
 'News']

In [105]:
##determine most popular categories
all_categories = df['category'].tolist()

In [107]:
stop_categories = ['#MemeOfTheWeek','All Things Considered','All Things Considered for June 10, 2016',
                   'All Things Considered for June 11, 2016','Classical Notes Newsletter','Code Switch',
                   'Goats and Soda','Home Page Top Stories','Latest From NPR News','MPX Break','MPX Invest',
                   'MPX Music interviews','MPX Open Door','MPX Parking Lot','MPX election stories ',
                   'Must Reads','Music Makers','NPR Selects','NPR Stories For Apple News',
                   "NPR.org's Most Popular Stories",'Policy-ish','Shots - Health News','Story of the Day',
                   'The Two-Way','Two-Way Featured Post Two',"Wait Wait...Don't Tell Me!",
                   "Wait Wait...Don't Tell Me! For June 11, 2016",'Weekend Edition Saturday',
                   'Weekend Edition Saturday for June 11, 2016','Weekend Edition Sunday',
                   'Weekend Edition Sunday for June 12, 2016','World Story of the Day','Your Health',
                   'nprreads','sunday puzzle','A No. 1']

In [108]:
all_categories_filtered = [x for x in all_categories if x not in stop_categories]

In [109]:
len(all_categories)

591

In [110]:
len(all_categories_filtered)

393

In [111]:
category_counter = {}  

In [112]:
for category in all_categories_filtered:
    if category in category_counter:
        category_counter[category] += 1
    else:
        category_counter[category] = 1

In [113]:
popular_categories = sorted(category_counter, key = category_counter.get, reverse = True)

In [114]:
top_5 = popular_categories[:5]

In [115]:
top_5

['News', 'U.S.', 'Around the Nation', 'World', 'Politics']

In [116]:
top_30 = popular_categories[:30]

In [117]:
top_30

['News',
 'U.S.',
 'Around the Nation',
 'World',
 'Politics',
 'America',
 'Arts & Life',
 'Business',
 'Health',
 'Music',
 'International',
 'Sports',
 'Books',
 'Global Health',
 'Environment',
 'Book Reviews',
 'Author Interviews',
 'Health Care',
 'Race',
 'Science',
 'Games & Humor',
 'Medical Treatments',
 'Donald Trump',
 'Remembrances',
 'Fiction',
 'Orlando',
 'Elections',
 'Africa',
 'Law',
 'Middle East']

In [118]:
##filter category lists to retain only top categories

In [119]:
for index, row in df_reshape.iterrows():
    compressed_category = [x for x in row['category'] if x in top_30]
    df_reshape['top_category'][index] = compressed_category

In [120]:
df_reshape.head()

id                                              title  \
0  481776539  Obama: 'An Attack On Any American ... Is An At...   
1  481766394       50 Dead, 53 Hospitalized In Orlando Shooting   
2  481702569      This Puzzle Packs Syllables Into Small Spaces   
3  481679101  To Fight Floundering Attendance, SeaWorld Turn...   
4  481620159  Latinos And American Identity In A Time Of Tru...   

                                            category  \
0  [News, Home Page Top Stories, mass shootings, ...   
1  [U.S., Weekend Edition Sunday for June 12, 201...   
2  [Games & Humor, Weekend Edition Sunday for Jun...   
3  [Business, Weekend Edition Sunday for June 12,...   
4  [Politics, News, Weekend Edition Sunday for Ju...   

                                        top_category  
0                     [News, Orlando, America, News]  
1              [U.S., Around the Nation, U.S., News]  
2                     [Games & Humor, Games & Humor]  
3  [Business, Around the Nation, Business, U.S., ...  
4  [Politics, News, Elections, Donald Trump, Amer...

In [121]:
##d3 export:  create a list of pairs of relationships to convert to json

pairs_d3 = []

In [122]:
for category_list in df_reshape['top_category']:
    for pair in itertools.combinations(category_list, 2):
        if pair not in pairs_d3 and pair[::-1] not in pairs_d3 and pair[0] <> pair[1]:
            pairs_d3.append(pair)

In [123]:
df_d3 = pd.DataFrame(pairs_d3, columns=('source', 'target'))

In [124]:
df_d3

source             target
0                  News            Orlando
1                  News            America
2               Orlando            America
3                  U.S.  Around the Nation
4                  U.S.               News
5     Around the Nation               News
6              Business  Around the Nation
7              Business               U.S.
8              Business               News
9              Politics               News
10             Politics          Elections
11             Politics       Donald Trump
12             Politics            America
13                 News          Elections
14                 News       Donald Trump
15            Elections       Donald Trump
16            Elections            America
17         Donald Trump            America
18               Africa      Global Health
19               Africa               News
20        Global Health               News
21         Book Reviews              Books
22         Book Reviews        Arts & Life
23                Books        Arts & Life
24             Politics  Around the Nation
25             Politics               U.S.
26                 U.S.            Orlando
27                 U.S.            America
28              Orlando  Around the Nation
29              America  Around the Nation
..                  ...                ...
101             America            Science
102   Around the Nation                Law
103                 Law        Environment
104                 Law            Science
105                 Law           Business
106                 Law               News
107         Environment            Science
108         Environment           Business
109             Science           Business
110              Sports      International
111       Global Health             Health
112       Global Health              World
113              Health              World
114           Elections               U.S.
115        Donald Trump               U.S.
116        Remembrances            America
117             America             Sports
118              Sports  Around the Nation
119              Sports           Business
120              Health           Business
121         Health Care           Business
122  Medical Treatments           Business
123         Environment      International
124         Environment              World
125             America      International
126             America              World
127       International            Science
128             Science              World
129               World           Business
130       International           Business

[131 rows x 2 columns]

In [125]:
df_d3.to_csv('links.csv')

In [129]:
nodes = pd.DataFrame()

In [130]:
nodes['node'] = top_30

In [131]:
nodes

node
0                 News
1                 U.S.
2    Around the Nation
3                World
4             Politics
5              America
6          Arts & Life
7             Business
8               Health
9                Music
10       International
11              Sports
12               Books
13       Global Health
14         Environment
15        Book Reviews
16   Author Interviews
17         Health Care
18                Race
19             Science
20       Games & Humor
21  Medical Treatments
22        Donald Trump
23        Remembrances
24             Fiction
25             Orlando
26           Elections
27              Africa
28                 Law
29         Middle East

In [132]:
nodes.to_csv('nodes.csv')